## IBM Data Science Capstone Project Final Notebook
#### Topic : Opening a new Bermese Restaurant in Toronto

In this notebook, I will be creating clusters to find the most suitable location to open an authentic Burmese restaurant in Toronto, Canada.

In [1]:
#importing necessary libraries
import requests
import pandas as pd

In [2]:
#scrapping neighborhoods in Canada
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')
else:
    print('Page download error. Error code: {}'.format(page.status_code))

Page download successful


In [3]:
df_html = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
df_html.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
#Drop the the rows on which the Borough is empty
df_html.dropna(subset=['Borough'], inplace=True)

In [5]:
#Check Neighborhood is empty but Borough exists
n_empty_neighborhood = df_html[df_html['Neighbourhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

Number of rows on which Neighborhood column is empty: 1


In [6]:
#Show which neighborhood is emtpy but Borough exists
df_html[df_html['Neighbourhood'].isna()]

,Postcode,Borough,Neighbourhood
9,M9A,Queen's Park,NaN


In [7]:
#Replace empty Neighborhood with Borough name and check again
df_html['Neighbourhood'].fillna(df_html['Borough'], inplace=True)
n_empty_neighborhood = df_html[df_html['Neighbourhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

Number of rows on which Neighborhood column is empty: 0


In [8]:
#Confirm that Queen's Park Neighborhood is not empty now:
df_html[df_html['Borough']=="Queen's Park"]

,Postcode,Borough,Neighbourhood
9,M9A,Queen's Park,Queen's Park


In [9]:
#Group by Postcode / Borough
df_postcodes = df_html.groupby(['Postcode','Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)])
df_postcodes.reset_index(inplace=True)
df_postcodes.head(5)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
#Drop the the rows on which the Borough is empty
df_html.dropna(subset=['Borough'], inplace=True)

In [11]:
#Check Neighborhood is empty but Borough exists
n_empty_neighborhood = df_html[df_html['Neighbourhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

Number of rows on which Neighborhood column is empty: 0


In [12]:
print('The shape of the dataset is:',df_postcodes.shape)

The shape of the dataset is: (103, 3)


In [14]:
#to make it easier, we will store this in csv format.
#Export to .CSV
df_postcodes.to_csv('Toronto_Postcodes.csv')

In [15]:
import numpy as np

Since GeoEncoder library is not working for me, I will use the csv file downloaded

In [16]:
#Read CSV file from link and load into dataframe
url_csv = 'http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(url_csv)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
#use the previously cleaned data
df_neighborhoods = pd.read_csv('Toronto_Postcodes.csv',index_col=[0])
df_neighborhoods.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [18]:
# Make sure both dataframes have the same 
df_coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_neighborhoods.rename(columns={'Postcode': 'PostalCode'}, inplace=True)

In [19]:
# Merge both datasets
df_neighborhoods_coordinates = pd.merge(df_neighborhoods, df_coordinates, on='PostalCode')
df_neighborhoods_coordinates.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [20]:
# Check coordinates for a couple of neighborhoods
df_neighborhoods_coordinates[(df_neighborhoods_coordinates['PostalCode']=='M5G') |
                             (df_neighborhoods_coordinates['PostalCode']=='M2H') ]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
17,M2H,North York,Hillcrest Village,43.803762,-79.363452
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [21]:
#Export to .CSV
df_neighborhoods_coordinates.to_csv('Toronto_Postcodes_2.csv')

In [22]:
import folium
from sklearn.cluster import KMeans

In [23]:
# Read .csv file from above
df = pd.read_csv('Toronto_Postcodes_2.csv', index_col=0)
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [24]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [25]:
df.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)

In [26]:
#count Bourough and Neighborhood
df.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           11
Mississauga          1
North York          24
Queen's Park         1
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [27]:
df_toronto = df[df['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
df_toronto.head()

/home/iankit/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3994: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [28]:
#Check the number of neighborhoods
print(df_toronto.groupby('Borough').count()['Neighborhood'])

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighborhood, dtype: int64


In [29]:
#Create list with the Boroughs (to be used later)
boroughs = df_toronto['Borough'].unique().tolist()

In [30]:
#Obtain the coordinates from the dataset itself, just averaging Latitude/Longitude of the current dataset 
lat_toronto = df_toronto['Latitude'].mean()
lon_toronto = df_toronto['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(lat_toronto, lon_toronto))

The geographical coordinates of Toronto are 43.66713498717948, -79.38987324871795


In [31]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3)) #Random color

In [33]:
map_toronto = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], 
                                           df_toronto['Longitude'],
                                           df_toronto['Borough'], 
                                           df_toronto['Neighborhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

## Getting Venues Data using Foursquare

In [34]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
#Get venues for all neighborhoods in our dataset
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                latitudes=df_toronto['Latitude'],
                                longitudes=df_toronto['Longitude'])

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [37]:
#Check size of resulting dataframe
toronto_venues.shape

(1714, 7)

In [38]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant


In [39]:
#Number of venues per neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton, Exhibition Place, Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",47,47,47,47,47,47
Central Bay Street,83,83,83,83,83,83
"Chinatown, Grange Park, Kensington Market",87,87,87,87,87,87
Christie,19,19,19,19,19,19


In [40]:
#Number of unique venue categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 230 uniques categories.


In [41]:
#print out the list of categories
toronto_venues['Venue Category'].unique()[:100]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Asian Restaurant', 'Greek Restaurant', 'Cosmetics Shop',
       'Italian Restaurant', 'Ice Cream Shop', 'Brewery', 'Yoga Studio',
       'Fruit & Vegetable Store', 'Dessert Shop', 'Pizza Place',
       'Bookstore', 'Restaurant', 'Juice Bar', 'Bubble Tea Shop', 'Diner',
       'Spa', 'Furniture / Home Store', 'Grocery Store', 'Coffee Shop',
       'Bakery', 'Caribbean Restaurant', 'Frozen Yogurt Shop',
       'American Restaurant', 'Liquor Store', 'Gym', 'Burger Joint',
       'Fish & Chips Shop', 'Park', 'Sushi Restaurant', 'Burrito Place',
       'Pet Store', 'Steakhouse', 'Fast Food Restaurant', 'Movie Theater',
       'Sandwich Place', 'Light Rail Station', 'Fish Market', 'Café',
       'Cheese Shop', 'Gay Bar', 'Seafood Restaurant',
       'Middle Eastern Restaurant', 'Comfort Food Restaurant',
       'Thai Restaurant', 'Stationery Store', 'Wine Bar',
       'Coworking Space', 'Bar', 'Latin American Restaurant',
  

In [42]:
# check if the results contain "Thai Restaurants"
#please note I changed the data to Thai because I was previously writing the code using Asian but the number is so small
"Thai Restaurant" in toronto_venues['Venue Category'].unique()

True

***Analyze Each Neighborhood***

In [43]:
# one hot encoding
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(1714, 231)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [44]:
to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(39, 231)


,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.020000,0.000000,...,0.000000,0.00000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.01,0.000000
1,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.00,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.043478
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.021277,0.000000,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012048,0.000000,...,0.000000,0.00000,0.00000,0.00,0.012048,0.000000,0.000000,0.012048,0.00,0.012048
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00,0.045977,0.000000,0.068966,0.011494,0.00,0.000000
8,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Church and Wellesley,0.012195,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012195,0.000000,...,0.012195,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.012195


In [45]:
len(to_grouped[to_grouped["Thai Restaurant"] > 0])

15

Create a new dataframe to find Asian Restaurants only

In [46]:
to_asian = to_grouped[["Neighborhoods","Thai Restaurant"]]

In [47]:
to_asian.head()

,Neighborhoods,Thai Restaurant
0,"Adelaide, King, Richmond",0.030000
1,Berczy Park,0.017857
2,"Brockton, Exhibition Place, Parkdale Village",0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000


## Cluster Neighborhoods

Run k-means to cluster the neighborhoods in Toronto into 3 clusters.

In [48]:
# set number of clusters
toclusters = 3

to_clustering = to_asian.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 2, 2, 0, 0, 0, 2, 0], dtype=int32)

In [49]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
to_merged = to_asian.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

In [50]:
to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
to_merged.head()

,Neighborhood,Thai Restaurant,Cluster Labels
0,"Adelaide, King, Richmond",0.030000,0
1,Berczy Park,0.017857,0
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,2
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,2
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,2


In [51]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
to_merged = to_merged.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")

print(to_merged.shape)
to_merged.head()

(1714, 9)


,Neighborhood,Thai Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Adelaide, King, Richmond",0.03,0,43.650571,-79.384568,Four Seasons Centre for the Performing Arts,43.650592,-79.385806,Concert Hall
0,"Adelaide, King, Richmond",0.03,0,43.650571,-79.384568,The Keg Steakhouse + Bar,43.649937,-79.384196,Steakhouse
0,"Adelaide, King, Richmond",0.03,0,43.650571,-79.384568,Nathan Phillips Square,43.652270,-79.383516,Plaza
0,"Adelaide, King, Richmond",0.03,0,43.650571,-79.384568,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant
0,"Adelaide, King, Richmond",0.03,0,43.650571,-79.384568,Shangri-La Toronto,43.649129,-79.386557,Hotel


In [52]:
# sort the results by Cluster Labels
print(to_merged.shape)
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged

(1714, 9)


,Neighborhood,Thai Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Adelaide, King, Richmond",0.03,0,43.650571,-79.384568,Four Seasons Centre for the Performing Arts,43.650592,-79.385806,Concert Hall
16,"First Canadian Place, Underground city",0.01,0,43.648429,-79.382280,Earls Kitchen & Bar,43.647946,-79.383706,Bar
16,"First Canadian Place, Underground city",0.01,0,43.648429,-79.382280,The National Club,43.649343,-79.380574,Wine Bar
16,"First Canadian Place, Underground city",0.01,0,43.648429,-79.382280,Sweet Lulu,43.650557,-79.381175,Asian Restaurant
16,"First Canadian Place, Underground city",0.01,0,43.648429,-79.382280,Indigospirit,43.648350,-79.380347,Bookstore
...,...,...,...,...,...,...,...,...,...
23,"Little Portugal, Trinity",0.00,2,43.647927,-79.419750,Pilot Coffee Roasters,43.646610,-79.419606,Coffee Shop
23,"Little Portugal, Trinity",0.00,2,43.647927,-79.419750,The Goods,43.649259,-79.424022,Vegetarian / Vegan Restaurant
23,"Little Portugal, Trinity",0.00,2,43.647927,-79.419750,The Tampered Press,43.650062,-79.417280,Coffee Shop
23,"Little Portugal, Trinity",0.00,2,43.647927,-79.419750,Trinity Bellwoods Park,43.647072,-79.413756,Park


Visualize the clusters

In [53]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [56]:
# create map
map_clusters = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i+x+(i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [57]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## Examine Clusters

In [58]:
#Cluster 0
to_merged.loc[to_merged['Cluster Labels'] == 0]

,Neighborhood,Thai Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Adelaide, King, Richmond",0.030000,0,43.650571,-79.384568,Four Seasons Centre for the Performing Arts,43.650592,-79.385806,Concert Hall
16,"First Canadian Place, Underground city",0.010000,0,43.648429,-79.382280,Earls Kitchen & Bar,43.647946,-79.383706,Bar
16,"First Canadian Place, Underground city",0.010000,0,43.648429,-79.382280,The National Club,43.649343,-79.380574,Wine Bar
16,"First Canadian Place, Underground city",0.010000,0,43.648429,-79.382280,Sweet Lulu,43.650557,-79.381175,Asian Restaurant
16,"First Canadian Place, Underground city",0.010000,0,43.648429,-79.382280,Indigospirit,43.648350,-79.380347,Bookstore
...,...,...,...,...,...,...,...,...,...
7,"Chinatown, Grange Park, Kensington Market",0.011494,0,43.653206,-79.400049,Kanto,43.652167,-79.404843,Filipino Restaurant
7,"Chinatown, Grange Park, Kensington Market",0.011494,0,43.653206,-79.400049,Krispy Kreme,43.655834,-79.399417,Donut Shop
7,"Chinatown, Grange Park, Kensington Market",0.011494,0,43.653206,-79.400049,Bun Saigon Vietnamese Restaurant,43.651676,-79.397656,Vietnamese Restaurant
7,"Chinatown, Grange Park, Kensington Market",0.011494,0,43.653206,-79.400049,Petit Nuage,43.651717,-79.403999,Dessert Shop


In [59]:
#Cluster 1
to_merged.loc[to_merged['Cluster Labels'] == 1]

,Neighborhood,Thai Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
21,"High Park, The Junction South",0.083333,1,43.661608,-79.464763,Indie Alehouse,43.665475,-79.465290,Gastropub
21,"High Park, The Junction South",0.083333,1,43.661608,-79.464763,Hole in the Wall,43.665296,-79.465118,Bar
21,"High Park, The Junction South",0.083333,1,43.661608,-79.464763,Lithuania Park,43.658667,-79.463038,Park
21,"High Park, The Junction South",0.083333,1,43.661608,-79.464763,famous last words,43.665181,-79.468471,Speakeasy
21,"High Park, The Junction South",0.083333,1,43.661608,-79.464763,Junction Flea,43.665258,-79.462868,Flea Market
21,"High Park, The Junction South",0.083333,1,43.661608,-79.464763,SMASH,43.665496,-79.465537,Antique Shop
21,"High Park, The Junction South",0.083333,1,43.661608,-79.464763,ARTiculations,43.665550,-79.467194,Arts & Crafts Store
21,"High Park, The Junction South",0.083333,1,43.661608,-79.464763,The Beet Organic Café,43.665340,-79.467137,Café
21,"High Park, The Junction South",0.083333,1,43.661608,-79.464763,Junction City Music Hall,43.665334,-79.466253,Music Venue
21,"High Park, The Junction South",0.083333,1,43.661608,-79.464763,Mjölk,43.665432,-79.467962,Furniture / Home Store


In [60]:
#Cluster 2
to_merged.loc[to_merged['Cluster Labels'] == 2]

,Neighborhood,Thai Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2,"Brockton, Exhibition Place, Parkdale Village",0.0,2,43.636847,-79.428191,Pure Yoga Toronto,43.637330,-79.423800,Yoga Studio
38,"The Danforth West, Riverdale",0.0,2,43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
38,"The Danforth West, Riverdale",0.0,2,43.679557,-79.352188,Athen's Pastries,43.678166,-79.348927,Greek Restaurant
38,"The Danforth West, Riverdale",0.0,2,43.679557,-79.352188,Book City,43.677413,-79.352734,Bookstore
38,"The Danforth West, Riverdale",0.0,2,43.679557,-79.352188,Il Fornello,43.678604,-79.346904,Italian Restaurant
...,...,...,...,...,...,...,...,...,...
23,"Little Portugal, Trinity",0.0,2,43.647927,-79.419750,Pilot Coffee Roasters,43.646610,-79.419606,Coffee Shop
23,"Little Portugal, Trinity",0.0,2,43.647927,-79.419750,The Goods,43.649259,-79.424022,Vegetarian / Vegan Restaurant
23,"Little Portugal, Trinity",0.0,2,43.647927,-79.419750,The Tampered Press,43.650062,-79.417280,Coffee Shop
23,"Little Portugal, Trinity",0.0,2,43.647927,-79.419750,Trinity Bellwoods Park,43.647072,-79.413756,Park


## Observations

Most of Thai restaurants are in Cluster 2 which is around Adelaide, King, Richmond areas and lowest (close to zero) in Cluster 1 areas which are North Toronto West and Parkdale areas. Also, there are good opportunities to open near Chinatown, St James town as the competition seems to be low. Looking at nearby venues, it seems Cluster 1 might be a good location as there are not a lot of Asian restaurants in these areas. Therefore, this project recommends the entrepreneur to open an authentic Burmese restaurant in these locations with little to no competition. Nonetheless, if the food is authentic, affordable and good taste, I am confident that it will have great following everywhere =)